In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

**9. We will now consider the Boston housing data set, from the MASS
library.
(a) Based on this data set, provide an estimate for the population
mean of medv . Call this estimate ˆ μ.**

In [3]:
Boston = pd.read_csv("data/Boston.csv")
Boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [4]:
Boston['medv'].mean()

22.532806324110677

**(b) Provide an estimate of the standard error of ˆ μ. Interpret this
result.
Hint: We can compute the standard error of the sample mean by
dividing the sample standard deviation by the square root of the
number of observations.**

In [5]:
Boston['medv'].std()/(len(Boston))**0.5

0.40886114749753505

相对与平均值22，标准误0.40886很小。

**(c) Now estimate the standard error of ˆ μ using the bootstrap. How
does this compare to your answer from (b)?**

In [6]:
#> boot(Portfolio ,alpha.fn,R=1000)定义对data根据statistic统计方式采样R次的生成数据，详见实验
def bootstrap_stats(data, statistic, R, random_state_start = 0):
    n_shape = len(data)
    try:
        statistic_original = list(statistic(data))
    except TypeError:
        statistic_original = [statistic(data)]
    statistic_list = []
    train_list = []
    statistic_len = len(statistic_original)
    for statistic_t in range(statistic_len):
        statistic_list.append([])
    for random_state in range(R):
        train = data.sample(n = n_shape,replace=True,random_state= random_state + random_state_start)
        try:
            float(statistic(train))
        except:
            try: 
                for statistic_factor in list(statistic(train)):
                    float(statistic_factor)
            except:
                continue
        train_list.append(train)
        for statistic_t in range(statistic_len):
            try:
                statistic_list[statistic_t].append(list(statistic(train))[statistic_t])
            except TypeError:
                statistic_list[statistic_t].append([statistic(train)][statistic_t])
    statistic_index = []
    statistic_mean = []
    statistic_std_error = []
    statistic_bias = []
    for statistic_t in range(statistic_len):
        statistic_index.append('t' + str(statistic_t) + '*')
        statistic_mean.append(np.mean(statistic_list[statistic_t]))
        statistic_std_error.append(((np.sum((statistic_list[statistic_t] - statistic_mean[statistic_t]) ** 2)) / (R - 1)) ** 0.5)
        statistic_bias.append(statistic_original[statistic_t] - statistic_mean[statistic_t])
    bootstrap_stats_pd = pd.DataFrame({'original':statistic_original,
                                       'mean':statistic_mean,
                                       'bias':statistic_bias,
                                       'std. error':statistic_std_error                                      
                                      },
                                     index = statistic_index)
    print(bootstrap_stats_pd)
    return statistic_list

In [7]:
def get_u_mean(data):
    return data['medv'].mean()

In [8]:
statistic_list = bootstrap_stats(Boston ,get_u_mean, R=1000)

      original       mean      bias  std. error
t0*  22.532806  22.528224  0.004583    0.426643


**（b）与（c）结果接近**

**(d) Based on your bootstrap estimate from (c), provide a 95% con-
fidence interval for the mean of medv . Compare it to the results
obtained using t.test(Boston$medv) .
Hint: You can approximate a 95% confidence interval using the
formula [ˆ μ − 2SE(ˆ μ), ˆ μ + 2SE(ˆ μ)].**

In [9]:
np.array([22.532806 - 2*0.426643 ,22.532806 + 2*0.426643])

array([21.67952 , 23.386092])

In [10]:
from scipy import stats
#求解置信区间stats.t.interval(置信度，自由度，均值，标准误)
stats.t.interval(0.95,len(Boston)-1,Boston['medv'].mean(),Boston['medv'].std()/(len(Boston))**0.5)

(21.729528014578595, 23.33608463364276)

**(e) Based on this data set, provide an estimate, ˆ μ med , for the median
value of medv in the population.**

In [11]:
Boston['medv'].median()

21.2

**(f) We now would like to estimate the standard error of ˆ μ med . Unfor-
tunately, there is no simple formula for computing the standard
error of the median. Instead, estimate the standard error of the
median using the bootstrap. Comment on your findings.**

In [12]:
def get_u_med(data):
    return data['medv'].median()

In [13]:
statistic_list = bootstrap_stats(Boston ,get_u_med, R=1000)

     original     mean    bias  std. error
t0*      21.2  21.1774  0.0226     0.39205


Median of 21.2 with SE of 0.39205. Small standard error relative to median value.

**(g) Based on this data set, provide an estimate for the tenth per-
centile of medv in Boston suburbs. Call this quantity ˆ μ 0.1 . (You
can use the quantile() function.)**

In [14]:
Boston['medv'].quantile(0.1)

12.75

**(h) Use the bootstrap to estimate the standard error of ˆ μ 0.1 . Com-
ment on your findings.**

In [15]:
def get_u_quantile(data):
    return data['medv'].quantile(0.1)

In [16]:
statistic_list = bootstrap_stats(Boston ,get_u_quantile, R=1000)

     original      mean     bias  std. error
t0*     12.75  12.77245 -0.02245    0.499267


Tenth-percentile of 12.75 with SE of 0.499. Small standard error relative to tenth-percentile value.